In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws,explode
import mysql.connector

# MySQL JDBC 드라이버 경로 (압축 푼 드라이버 JAR 파일 경로)
mysql_driver_path = "/home/ubuntu/mysql-connector-j-9.2.0/mysql-connector-j-9.2.0.jar"  # 다운로드한 JAR 파일의 경로를 설정해주세요.

# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS JSON File Reading") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.jars", mysql_driver_path) \
    .getOrCreate()

In [7]:
# HDFS에서 JSON 파일 읽기
file_path = "hdfs://localhost:9000/shared_data/label_data/1.Car/1.horn_of_car"
df = spark.read.json(file_path, multiLine=True)

In [8]:
df.printSchema()

root
 |-- annotations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- area: struct (nullable = true)
 |    |    |    |-- end: double (nullable = true)
 |    |    |    |-- start: double (nullable = true)
 |    |    |-- categories: struct (nullable = true)
 |    |    |    |-- category_01: string (nullable = true)
 |    |    |    |-- category_02: string (nullable = true)
 |    |    |    |-- category_03: string (nullable = true)
 |    |    |-- decibel: long (nullable = true)
 |    |    |-- labelName: string (nullable = true)
 |    |    |-- soundQuality: string (nullable = true)
 |    |    |-- subCategory: string (nullable = true)
 |-- audio: struct (nullable = true)
 |    |-- bitRate: string (nullable = true)
 |    |-- duration: double (nullable = true)
 |    |-- fileFormat: string (nullable = true)
 |    |-- fileName: string (nullable = true)
 |    |-- fileSize: long (nullable = true)
 |    |-- recodingType: string (nullable = true)
 |    |-- sample

In [24]:
# SQL 쿼리로 데이터 추출
df.createOrReplaceTempView("json_table")

result_df = spark.sql("""
    SELECT  
        json_table.audio.duration, 
        json_table.audio.recodingType, 
        exploded_annotations.area.start AS area_start,
        exploded_annotations.area.end AS area_end
    FROM json_table
    LATERAL VIEW explode(json_table.annotations) AS exploded_annotations
    LIMIT 50
""")

In [25]:
# 추출된 데이터 출력
result_df.show(10, truncate=False)

+--------+------------+----------+--------+
|duration|recodingType|area_start|area_end|
+--------+------------+----------+--------+
|11.584  |Mono        |9.99      |10.55   |
|22.4    |Mono        |16.05     |16.62   |
|9.401   |Stereo      |2.0       |7.401   |
|7.178   |Stereo      |2.68      |4.13    |
|14.944  |Mono        |2.0       |12.944  |
|8.733   |Stereo      |3.11      |5.48    |
|10.539  |Mono        |2.0       |8.539   |
|10.557  |Mono        |2.0       |8.557   |
|10.581  |Mono        |2.0       |8.581   |
|5.884   |Stereo      |2.0       |3.884   |
+--------+------------+----------+--------+
only showing top 10 rows



In [26]:
# MySQL 연결 정보
mysql_url = "jdbc:mysql://15.168.145.74:3306/my_db?useUnicode=true&characterEncoding=UTF-8"
mysql_properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [27]:
# MySQL로 DataFrame 적재 (쿼리 결과가 None이 아닌 경우에만)
if result_df is not None:
    result_df.write.jdbc(url=mysql_url, table="car_horn_data", mode="overwrite", properties=mysql_properties)
    print("데이터가 MySQL로 성공적으로 적재되었습니다!")
else:
    print("쿼리 결과가 없습니다. 데이터 추출이 실패했습니다.")

데이터가 MySQL로 성공적으로 적재되었습니다!
